In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import ipywidgets as widgets
from functools import partial
from wrwc.config import RAW_DATA_DIR, PROCESSED_DATA_DIR

2025-07-03 10:45:06.860 | INFO     | wrwc.config:<module>:11 - PROJ_ROOT path is: /Users/gdang2/repos/wrwc-water-quality


## I. Data Processing and Shape

In [2]:
df_data_raw = pd.read_csv(RAW_DATA_DIR / 'WoonasquatucketData.csv')
df_data = df_data_raw.copy()

# Standardize column names
df_data.columns = [col.replace(" ", "_").lower() for col in df_data.columns]

# Create dictionary of parameter codes to parameter name
param_code_to_name = {s.split('-')[-1].strip(): s.split('-')[0].strip() for s in df_data.parameter.unique()}
shortened_names = {'00915': 'Calcium',
                   '32209': 'Chlorophyll a',
                   '82903': 'Depth',
                   '00631': 'Nitrate + Nitrite',
                   '00930': 'Sodium',
                   '00600': 'Nitrogen, Total',
                   '00608': 'Nitrogen, Ammonia',
                  }
param_code_to_name.update(shortened_names)

# Read Site info 
df_site = pd.read_csv(RAW_DATA_DIR / 'SiteInfo.csv')
df_site.columns = [col.replace(" ", "_").lower() for col in df_site.columns]
df_site.rename(columns={'ww_station': 'ww_id'}, inplace=True)


# Process dataframe
df_data = (
    df_data
    # Create datetime, parameter code, and parameter name columns
    .assign(date=pd.to_datetime(df_data['date_of_sample']),
            param_code=[s.split('-')[-1].strip() for s in df_data['parameter']])
    .assign(parameter=lambda x: [param_code_to_name[s] for s in x['param_code']])
    # Drop no data or redundant columns
    .dropna(subset='concentration')
    .drop(columns=['sediment_particle_size', 'particle_size_unit', 'fish_sample_type', 'fish_taxa', 'date_of_sample'])
    .merge(df_site.loc[:, ['ww_id', 'wbid', 'wb_type', 'site_descr', 'lat_dd', 'lon_dd']], on='ww_id', how='left')
    .set_index('date')
)


df_data.head()

,source.name,ww_id,time,sample_type,sample_media,depth,parameter,concentration,unit,qualifier_code,...,comments,monitoring_location,watershed,watershed_code,param_code,wbid,wb_type,site_descr,lat_dd,lon_dd
date,,,,,,,,,,,,,,,,,,,,,
1990-04-28,URIWW-1990-Lakes.csv,WW016,12:00:00 PM,Composite,Water,NaN,Secchi Depth,2.025,m,NaN,...,NaN,Geo @ Widow Sweet Rd,Woonasquatucket River,WO,00069,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331
1990-05-05,URIWW-1990-Lakes.csv,WW016,10:30:00 AM,Grab,Water,1.0,Alkalinity,8.700,mg/l,NaN,...,NaN,Geo @ Widow Sweet Rd,Woonasquatucket River,WO,00410,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331
1990-05-05,URIWW-1990-Lakes.csv,WW016,10:30:00 AM,Grab,Water,1.0,Calcium,5.930,mg/l,NaN,...,NaN,Geo @ Widow Sweet Rd,Woonasquatucket River,WO,00915,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331
1990-05-05,URIWW-1990-Lakes.csv,WW016,10:30:00 AM,Grab,Water,1.0,Chloride,36.000,mg/l,NaN,...,NaN,Geo @ Widow Sweet Rd,Woonasquatucket River,WO,00940,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331
1990-05-05,URIWW-1990-Lakes.csv,WW016,10:30:00 AM,Grab,Water,1.0,Chlorophyll a,4.200,ug/l,NaN,...,Analog,Geo @ Widow Sweet Rd,Woonasquatucket River,WO,32209,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331


In [3]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25620 entries, 1990-04-28 to 2021-10-23
Data columns (total 26 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   source.name               25620 non-null  object 
 1   ww_id                     25620 non-null  object 
 2   time                      10720 non-null  object 
 3   sample_type               25620 non-null  object 
 4   sample_media              25620 non-null  object 
 5   depth                     20916 non-null  float64
 6   parameter                 25620 non-null  object 
 7   concentration             25620 non-null  float64
 8   unit                      25620 non-null  object 
 9   qualifier_code            1155 non-null   object 
 10  detection_limit           25619 non-null  float64
 11  detection_limit_unit      25620 non-null  object 
 12  quantitation_level        24613 non-null  float64
 13  quantitation_level_unit   25620 non-null  ob

---
## II. Sample Counts
Constructing table and heatmaps of sample counts by parameter and site.

In [4]:
# Heatmap function 
def heatmap(df, title):
    fig = go.Figure(data=go.Heatmap(
        z=df.values,
        x=df.columns,
        y=df.index,
        ))

    fig.update_layout(
        title=dict(text=title),
        xaxis_nticks=32)
    fig.update_xaxes(tickangle=90)
    
    fig.show()

# Calculate monthly counts by site and parameter 
df_counts = (
    df_data
    .groupby(['parameter', 'wb_type', 'ww_id'])
    .resample("MS")
    .size()
    .unstack(fill_value=0)
)
df_counts

date                              1990-04-01  1990-05-01  1990-06-01  \
parameter  wb_type         ww_id                                       
Alkalinity Lake or Pond    WW201           0           0           0   
           Reservoir       WW016           0           1           0   
                           WW024           0           0           0   
                           WW046           0           0           0   
                           WW052           0           1           0   
...                                      ...         ...         ...   
pH         Stream or river WW240           0           0           0   
                           WW241           0           0           0   
                           WW635           0           0           0   
                           WW679           0           0           0   
                           WW680           0           0           0   

date                              1990-07-01  1990-08-01  1990-09-01  \
parameter  wb_type         ww_id                                       
Alkalinity Lake or Pond    WW201           0           0           0   
           Reservoir       WW016           1           0           0   
                           WW024           0           0           0   
                           WW046           1           0           0   
                           WW052           1           0           0   
...                                      ...         ...         ...   
pH         Stream or river WW240           0           0           0   
                           WW241           0           0           0   
                           WW635           0           0           0   
                           WW679           0           0           0   
                           WW680           0           0           0   

date                              1990-10-01  1990-11-01  1990-12-01  \
parameter  wb_type         ww_id                                       
Alkalinity Lake or Pond    WW201           0           0           0   
           Reservoir       WW016           0           1           0   
                           WW024           0           0           0   
                           WW046           0           1           0   
                           WW052           0           0           0   
...                                      ...         ...         ...   
pH         Stream or river WW240           0           0           0   
                           WW241           0           0           0   
                           WW635           0           0           0   
                           WW679           0           0           0   
                           WW680           0           0           0   

date                              1991-01-01  ...  2021-01-01  2021-02-01  \
parameter  wb_type         ww_id              ...                           
Alkalinity Lake or Pond    WW201           0  ...           0           0   
           Reservoir       WW016           0  ...           0           0   
                           WW024           0  ...           0           0   
                           WW046           0  ...           0           0   
                           WW052           0  ...           0           0   
...                                      ...  ...         ...         ...   
pH         Stream or river WW240           0  ...           0           0   
                           WW241           0  ...           0           0   
                           WW635           0  ...           0           0   
                           WW679           0  ...           0           0   
                           WW680           0  ...           0           0   

date                              2021-03-01  2021-04-01  2021-05-01  \
parameter  wb_type         ww_id                                       
Alkalinity Lake or Pond    WW201           

### Heatmap Widget

In [5]:
# Set up dropdown widgets
parameter = widgets.Dropdown(
    options=sorted(df_data.parameter.unique()),
    description='Parameter:',
    disabled=False,
)

site_type = widgets.Dropdown(
    options=sorted(df_data.query(f"parameter == '{parameter.value}'")["wb_type"].unique()),
    description='Site Type:',
    disabled=False,
)

# Callback to update site_type options based on selected parameter 
def handle_parameter_change(change):
    site_type.options = sorted(df_data.query(f"parameter == '{parameter.value}'")["wb_type"].unique())

# Plotting callback 
def heatmap_selector(parameter, site_type):
    heatmap(df_counts.loc[parameter, site_type], f'{parameter} Counts by Site')

# Observe changes in parameter to update site types 
parameter.observe(handle_parameter_change, names="value")

# Set up interactive widgets 
widgets.interact(heatmap_selector, parameter=parameter, site_type=site_type)

interactive(children=(Dropdown(description='Parameter:', options=('Alkalinity', 'Calcium', 'Chloride', 'Chloro…

<function __main__.heatmap_selector(parameter, site_type)>

In [6]:
# Summary table of counts 
# The table we will group by site, years, parameter, depth
df_param_count = df_data.groupby(['ww_id', df_data.index.year, 'parameter', 'depth']).size()

In [7]:
pd.set_option('display.max_rows', 1000)
print(df_param_count['WW016'])
pd.set_option('display.max_rows', 100)

date  parameter              depth
1990  Alkalinity             1.0       3
      Calcium                1.0       3
      Chloride               1.0       3
      Chlorophyll a          1.0      13
                             5.0       3
      Dissolved Oxygen       1.0       2
                             5.0      13
      Magnesium              1.0       3
      Nitrate + Nitrite      1.0       3
      Phosphorus, Dissolved  1.0       3
      Phosphorus, Total      1.0       3
      Sodium                 1.0       3
      Temperature            1.0      14
                             5.0      14
      pH                     1.0       3
1991  Alkalinity             1.0       3
      Calcium                1.0       3
      Chloride               1.0       3
      Chlorophyll a          1.0      14
                             5.0       3
      Dissolved Oxygen       1.0       3
                             5.0      14
      E.coli                 0.5       3
      Magnesium       

# III. Mapping file construction 
Make a CSV to import into GIS. 
* Each site will be a point 
* Attributes: Site code, Site name, Years measured, Parameters measured, Depths Measured, X, Y 

In [8]:
df_mapping = (
    df_data
    .reset_index()
    .groupby(['ww_id'])
    .agg({
        'parameter': lambda x: sorted(x.unique()),
        'date': lambda x: list(x.dt.year.unique()),
        'depth': lambda x: sorted(x.dropna().unique()),
}))
df_mapping = (df_mapping
              .merge(df_site.loc[:, ['ww_id', 'wbid', 'wb_type', 'site_descr', 'lat_dd', 'lon_dd']], 
                     left_index=True, 
                     right_on='ww_id', 
                     how='left'
                    )
              .loc[:, ['ww_id', 'wbid', 'wb_type', 'site_descr', 'lat_dd', 'lon_dd', 'parameter', 'date', 'depth']]
              .rename(columns={'parameter': 'parameters', 'date': 'years', 'depth': 'depths'})
             )
df_mapping

,ww_id,wbid,wb_type,site_descr,lat_dd,lon_dd,parameters,years,depths
0,WW016,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331,"[Alkalinity, Calcium, Chloride, Chlorophyll a,...","[1990, 1991, 1992, 1993, 1994, 1995, 1996, 199...","[0.1, 0.5, 1.0, 3.0, 4.0, 4.5, 5.0, 5.2, 5.5, ..."
1,WW024,RI0002007L-06,Reservoir,Lower Sprague Reservoir,41.887806,-71.548376,"[Alkalinity, Calcium, Chloride, Chlorophyll a,...","[1994, 1995, 1996, 1997, 1998, 2000, 2001, 200...","[0.5, 1.0]"
2,WW046,RI0002007L-03,Reservoir,Slack's Reservoir,41.859528,-71.551750,"[Alkalinity, Calcium, Chloride, Chlorophyll a,...","[1990, 1991, 1992, 1994, 1995, 1996, 1997, 199...","[0.0, 0.1, 0.5, 1.0, 2.0, 3.0, 4.0, 8.0]"
3,WW052,RI0002007L-07,Reservoir,Stillwater Pond,41.908342,-71.527331,"[Alkalinity, Calcium, Chloride, Chlorophyll a,...","[1990, 1991, 1992, 1994, 1996, 1997, 1998, 199...","[0.0, 0.5, 1.0, 3.0]"
4,WW061,RI0002007L-04,Reservoir,Waterman Reservoir,41.881519,-71.589489,"[Alkalinity, Calcium, Chloride, Chlorophyll a,...","[1990, 1991, 1992, 1993, 1994, 1995, 1996, 199...","[0.0, 0.1, 0.5, 1.0, 2.0]"
5,WW065,RI0002007L-08,Reservoir,Woonasquatucket Reservoir (Stump Pond),41.906078,-71.544722,"[Alkalinity, Calcium, Chloride, Chlorophyll a,...","[1990, 1991, 1992, 1993, 1994, 1995, 1996, 199...","[0.0, 0.5, 1.0, 3.0]"
6,WW113,RI0002007R-10A,Reservoir,Georgiaville @ Capron Pond/Woonasquatucket River,41.902792,-71.521688,"[Chloride, E.coli, Enterococci, Fecal Coliform...","[1995, 1996, 1997, 1998, 1999, 2000, 2001, 200...",[0.2]
7,WW114,RI0002007R-03,Stream or river,Georgiaville @ Harris/Harris Brook,41.899054,-71.510030,"[Chloride, E.coli, Enterococci, Fecal Coliform...","[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...",[0.2]
8,WW123,RI0002007R-15,Stream or river,Slack's Tributary 1 (A - Northwest corner - cu...,41.866375,-71.558561,"[Chloride, E.coli, Enterococci, Fecal Coliform...","[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...","[0.0, 0.2]"
9,WW124,RI0002007R-15,Stream or river,Slack's Tributary 2 (B - SW cove - drainage fr...,41.859348,-71.557825,"[Alkalinity, Chloride, E.coli, Enterococci, Fe...","[1996, 1997, 1998, 1999, 2000, 2001, 2002, 200...","[0.0, 0.2]"


In [9]:
def list_to_string(l: list, wrap: int = 4):
    output_str = ''
    l_strings = [str(s).replace(',', '') for s in l]
    for i, item in enumerate(l_strings):
        output_str += f'{item}'
        if i+1 != len(l_strings):
            output_str += ', '
            if (i+1)%wrap == 0:
                output_str += '<br>'

    return output_str

# Convert lists to strings for maping display
df_mapping_out = (
    df_mapping
    .assign(
        parameters=df_mapping['parameters'].apply(list_to_string),
        years=df_mapping['years'].apply(partial(list_to_string, wrap=8)),
        depths=df_mapping['depths'].apply(partial(list_to_string, wrap=10))
    ))
df_mapping_out

,ww_id,wbid,wb_type,site_descr,lat_dd,lon_dd,parameters,years,depths
0,WW016,RI0002007L-02,Reservoir,Georgiaville Pond,41.892783,-71.511331,"Alkalinity, Calcium, Chloride, Chlorophyll a, ...","1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997...","0.1, 0.5, 1.0, 3.0, 4.0, 4.5, 5.0, 5.2, 5.5, 5..."
1,WW024,RI0002007L-06,Reservoir,Lower Sprague Reservoir,41.887806,-71.548376,"Alkalinity, Calcium, Chloride, Chlorophyll a, ...","1994, 1995, 1996, 1997, 1998, 2000, 2001, 2002...","0.5, 1.0"
2,WW046,RI0002007L-03,Reservoir,Slack's Reservoir,41.859528,-71.551750,"Alkalinity, Calcium, Chloride, Chlorophyll a, ...","1990, 1991, 1992, 1994, 1995, 1996, 1997, 1998...","0.0, 0.1, 0.5, 1.0, 2.0, 3.0, 4.0, 8.0"
3,WW052,RI0002007L-07,Reservoir,Stillwater Pond,41.908342,-71.527331,"Alkalinity, Calcium, Chloride, Chlorophyll a, ...","1990, 1991, 1992, 1994, 1996, 1997, 1998, 1999...","0.0, 0.5, 1.0, 3.0"
4,WW061,RI0002007L-04,Reservoir,Waterman Reservoir,41.881519,-71.589489,"Alkalinity, Calcium, Chloride, Chlorophyll a, ...","1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997...","0.0, 0.1, 0.5, 1.0, 2.0"
5,WW065,RI0002007L-08,Reservoir,Woonasquatucket Reservoir (Stump Pond),41.906078,-71.544722,"Alkalinity, Calcium, Chloride, Chlorophyll a, ...","1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997...","0.0, 0.5, 1.0, 3.0"
6,WW113,RI0002007R-10A,Reservoir,Georgiaville @ Capron Pond/Woonasquatucket River,41.902792,-71.521688,"Chloride, E.coli, Enterococci, Fecal Coliform,...","1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002...",0.2
7,WW114,RI0002007R-03,Stream or river,Georgiaville @ Harris/Harris Brook,41.899054,-71.510030,"Chloride, E.coli, Enterococci, Fecal Coliform,...","1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003...",0.2
8,WW123,RI0002007R-15,Stream or river,Slack's Tributary 1 (A - Northwest corner - cu...,41.866375,-71.558561,"Chloride, E.coli, Enterococci, Fecal Coliform,...","1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003...","0.0, 0.2"
9,WW124,RI0002007R-15,Stream or river,Slack's Tributary 2 (B - SW cove - drainage fr...,41.859348,-71.557825,"Alkalinity, Chloride, E.coli, Enterococci, <br...","1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003...","0.0, 0.2"


In [10]:
# df_mapping_out.to_csv(PROCESSED_DATA_DIR / 'site_summary_20250424.csv', index=False)